# T3 - Running scenarios

While running individual sims can be interesting for simple explorations, at some point it will almost always be necessary to run a large number of simulations simultaneously -- to explore different scenarios, to perform calibration, or simply to get uncertainty bounds on a single projection. This tutorial explains how to do that.

虽然运行单个模拟对于简单的探索可能很有趣，但在某些情况下，几乎总是有必要同时运行大量模拟--探索不同的情景、执行校准或只是为了获得单个预测的不确定性边界。本教程将介绍如何做到这一点。

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/institutefordiseasemodeling/covasim/HEAD?urlpath=lab%2Ftree%2Fdocs%2Ftutorials%2Ftut_running.ipynb) to open an interactive version of this notebook.
    
</div>

## Running with MultiSims

The most common way to run multiple simulations is with the *MultiSim* object. As the name suggests, this is a relatively simple container for a number of sims. However, it contains powerful methods for plotting, statistics, and running all the sims in parallel.

运行多个模拟的最常用方法是使用 *MultiSim* 对象。顾名思义，这是一个相对简单的容器，可容纳多个模拟。不过，它包含了强大的绘图、统计和并行运行所有模拟的方法。

### Running one sim with uncertainty

Making and running a multisim based on a single sim is pretty easy:

制作和运行基于单个模拟的多重模拟非常简单：

In [ ]:
import covasim as cv
cv.options(jupyter=True, verbose=0)

sim = cv.Sim()
msim = cv.MultiSim(sim)
msim.run(n_runs=5)
msim.plot()

If you run a multisim with a single sim input as above, it will change the random seed for each sim, which is what leads to the variability you see.

By default, the multisim simply plots each simulation. These simulations are stored in the `sims` attribute, which is just a simple list of sims:

如果使用上述单一模拟输入运行多重模拟，它将改变每次模拟的随机种子，从而导致您看到的变化。

默认情况下，multisim 只会绘制每次模拟。这些模拟存储在 `sims` 属性中，它只是一个简单的模拟列表：

In [ ]:
for sim in msim.sims:
    sim.brief()

However, often (especially when you run the same parameters with different random seeds), you don't care about the individual sims, you want to see the *statistics* for the sims. You can calculate either the mean or the median of the results across all the sims as follows:

不过，通常情况下（尤其是在使用不同随机种子运行相同参数时），您并不关心单个模拟结果，而是想查看模拟结果的*统计数据。您可以计算所有模拟结果的平均值或中位数，方法如下：

In [ ]:
msim.mean()
msim.plot_result('new_infections')

In [ ]:
msim.median()
msim.plot_result('new_infections')

You can see these are similar, but slightly different. You can also treat each of the individual sims as part of a larger single sim, and "combine" the results into one sim:

您可以看到这些结果相似，但又略有不同。您也可以将每个单独的模拟视为更大的单一模拟的一部分，并将结果 “合并 ”到一个模拟中：

In [ ]:
msim.combine()
msim.plot_result('new_infections')

Note how now there is no uncertainty and the total number of infections is 5x higher than in the previous plots, since we just added 5 different sims together.

Each of these operations modifies the `msim.base_sim` object, and does not affect the actual list of stored sims, which is why you can go back and forth between them.

请注意，由于我们只是将 5 个不同的模拟添加到了一起，所以现在没有不确定性，感染总数也比之前的图多了 5 倍。

这些操作中的每一个都会修改 msim.base_sim 对象，而不会影响实际存储的模拟列表，因此您可以在它们之间来回切换。

### Running different sims

Often you don't want to run the same sim with different seeds, but instead want to run a set of different sims. That's also very easy -- for example, here's how you would do a sweep across beta values:

通常情况下，您不想用不同的种子运行同一个模拟，而是想运行一组不同的模拟。这也非常简单，例如，您可以在下面的表格中对不同的贝塔值进行扫描：

In [ ]:
import numpy as np

betas = np.linspace(0.010, 0.020, 5) # Sweep beta from 0.01 to 0.02 with 5 values
sims = []
for beta in betas:
    sim = cv.Sim(beta=beta, label=f'Beta = {beta}')
    sims.append(sim)
msim = cv.MultiSim(sims)
msim.run()
msim.plot_result('cum_infections')

As you would expect, higher beta values have more infections.

Finally, note that you can use multisims to do very compact scenario explorations -- here we are using the command `cv.parallel()`, which is an alias for `cv.MultiSim().run()`:

如你所料，β 值越高，感染越多。

最后，请注意您可以使用 multisims 进行非常紧凑的场景探索--这里我们使用的是 `cv.parallel()`命令，它是 `cv.MultiSim().run()`的别名：

In [ ]:
def protect_elderly(sim):
    if sim.t == sim.day('2021-04-01'):
        elderly = sim.people.age>70
        sim.people.rel_sus[elderly] = 0.0

pars = {'start_day':'2021-03-01', 'n_days':120}
s1 = cv.Sim(pars, label='Default')
s2 = cv.Sim(pars, label='Protect the elderly', interventions=protect_elderly)
cv.parallel(s1, s2).plot(['cum_deaths', 'cum_infections'])

<div class="alert alert-warning">

**Gotcha:** Because `multiprocess` pickles the sims when running them, `sims[0]` (before being run by the multisim) and `msim.sims[0]` are **not** the same object. After calling `msim.run()`, always use sims from the multisim object, not from before. In contrast, if you *don't* run the multisim (e.g. if you make a multisim from already-run sims), then `sims[0]` and `msim.sims[0]` are indeed exactly the same object.

**问题：** 由于 `multiprocess` 在运行模拟时会剔除模拟，因此 `sims[0]`（在被多重模拟运行之前）和 `msim.sims[0]` 并非**同一个对象。调用 `msim.run()`后，应始终使用 multisim 对象中的 sims，而不是之前的 sims。相反，如果**不运行多重模拟（例如，使用已运行的 sims 制作多重模拟），则 `sims[0]` 和 `msim.sims[0]` 确实是完全相同的对象。

</div>

### Advanced usage

Finally, you can also merge or split different multisims together. Here's an example that's similar to before, except it shows how to run a multisim of different seeds for the same beta value, but then merge multisims for different beta values together into one multisim to rule them all:

最后，你还可以合并或拆分不同的多重模拟。下面的示例与之前的类似，但它展示了如何针对相同的贝塔值运行不同种子的多重模拟，然后将不同贝塔值的多重模拟合并到一个多重模拟中，对它们进行排序：

In [ ]:
n_sims = 3
betas = [0.012, 0.016, 0.018]

msims = []
for beta in betas:
    sims = []
    for s in range(n_sims):
        sim = cv.Sim(pop_size=10e3, beta=beta, rand_seed=s, label=f'Beta = {beta}')
        sims.append(sim)
    msim = cv.MultiSim(sims)
    msim.run()
    msim.mean()
    msims.append(msim)

merged = cv.MultiSim.merge(msims, base=True)
merged.plot(color_by_sim=True)

As you can see, running this way lets you run not just different values, but run different values with uncertainty. Which brings us to...

正如您所看到的，这种运行方式不仅可以运行不同的值，还可以运行具有不确定性的不同值。这就给我们带来了...

## Running with Scenarios

Most of the time, you'll want to run with multisims since they give you the most flexibility. However, in certain cases, Scenario objects let you achieve the same thing more simply. Unlike MultiSims, which are completely agnostic about what sims you include, scenarios always start from the same base sim. They then modify the parameters as you specify, and finally add uncertainty if desired. For example, this shows how you'd use scenarios to run the example similar to the one above.

大多数情况下，你都希望使用多重假设运行，因为它们能给你带来最大的灵活性。但在某些情况下，情景对象可以让您更简单地实现同样的目标。多重模拟完全不考虑包含哪些模拟，而场景则不同，它总是从相同的基础模拟开始。然后，它们会根据您的指定修改参数，最后根据需要添加不确定性。例如，下面展示了如何使用情景模拟来运行与上面类似的示例。

In [ ]:
# Set base parameters -- these will be shared across all scenarios
basepars = {'pop_size':10e3} 

# Configure the settings for each scenario
scenarios = {'baseline': {
              'name':'Baseline',
              'pars': {}
              },
            'high_beta': {
              'name':'High beta (0.020)',
              'pars': {
                  'beta': 0.020,
                  }
              },
            'low_beta': {
              'name':'Low beta (0.012)',
              'pars': {
                  'beta': 0.012,
                  }
              },
             }

# Run and plot the scenarios
scens = cv.Scenarios(basepars=basepars, scenarios=scenarios)
scens.run()
scens.plot()